이름: 김수현

학번: 20215124

# Neural networks with PyTorch

Pytorch의 `nn.module`을 활용하여 만드는 유용한 방법을 학습합니다.

<div style="text-align:center"><img src='https://drive.google.com/thumbnail?id=1J2SeiPpVJs1-ML2BdLrcxkGGmHpRxIVE&sz=w1000' width="250" height="200">

### Lego block coding! </div>

In [86]:
import torch
import numpy as np

from torch import nn
import torch.nn.functional as F
from collections import OrderedDict

torch.manual_seed(0)

`nn.Linear`: $Z^{[\ell]} = A^{[\ell-1]}W^T+b$
연산.

해당 layer의

- 입력 차원 `n_input=30`
- 출력 차원 `n_output=60`


In [87]:
# Example of nn.linear
linear_layer1 = nn.Linear(30, 60)

In [88]:
A = torch.randn(60, 30)
linear_layer1(A).shape

torch.Size([60, 60])

How to get the weights and bias of each `nn.Linear`

In [89]:
# Example of weights
linear_layer1.weight.data = torch.ones_like(linear_layer1.weight)

In [90]:
linear_layer1.weight

Parameter containing:
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], requires_grad=True)

### NN example

- input units: 20
- hidden layer: 30, 40
- output units: 3
- activation function: ReLU
- output layer: No activation



In [91]:
# Simple NN construction

class FCN(nn.Module):
    def __init__(self):
        super().__init__()

        self.lin1 = nn.Linear(20, 30)
        self.lin2 = nn.Linear(30, 40)
        self.lin3 = nn.Linear(40, 3)
        self.relu = nn.ReLU(True)

    def forward(self, x):
        x = self.lin1(x)
        x = self.relu(x)
        x = self.lin2(x)
        x = self.relu(x)
        x = self.lin3(x)
        return x

In [92]:
Xtrain = torch.randn(60, 20)

model = FCN()
# model(Xtrain)

In [93]:
model

FCN(
  (lin1): Linear(in_features=20, out_features=30, bias=True)
  (lin2): Linear(in_features=30, out_features=40, bias=True)
  (lin3): Linear(in_features=40, out_features=3, bias=True)
  (relu): ReLU(inplace=True)
)

In [94]:
# Example of parameters() in models
# param_iterator = model.parameters()

# for param in param_iterator:
#     print(param)


In [95]:
# nn.Sequential() example

class FCN_seq(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc = nn.Sequential(nn.Linear(20, 30),
                      nn.ReLU(True),
                      nn.Linear(30, 40),
                      nn.ReLU(True),
                      nn.Linear(40, 3)
        )

    def forward(self, x):
        return self.fc(x)



# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [96]:
model_seq = FCN_seq()
Xtrain.shape
model_seq

FCN_seq(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)

In [97]:
model_seq.fc[0]

Linear(in_features=20, out_features=30, bias=True)

In [98]:
class FCN_seq_v2(nn.Module):
    def __init__(self):
        super().__init__()



        temp = self.fcn_block(20, 30)+self.fcn_block(30, 40)+[nn.Linear(40,1)]
        self.fc = nn.Sequential(*temp)


    def fcn_block(self, in_dim, out_dim):
        return [nn.Linear(in_dim, out_dim),
                             nn.ReLU(True)]


    def forward(self, x):
        return self.fc(x)



# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [99]:
model_seq_v2 = FCN_seq_v2()
model_seq_v2

FCN_seq_v2(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=1, bias=True)
  )
)

In [100]:
class FCN_final(nn.Module):
    def __init__(self, in_dim, hlayer, out_dim):
        super().__init__()

        l_list = self.fcn_block(in_dim, hlayer[0])

        for l1, l2 in zip(hlayer[:-1], hlayer[1:]):
            l_list = l_list + self.fcn_block(l1, l2)

        l_list = l_list + [nn.Linear(hlayer[-1], out_dim)]

        self.fc = nn.Sequential(*l_list)


    def fcn_block(self, in_dim, out_dim):
        return [nn.Linear(in_dim, out_dim),
                             nn.ReLU(True)]


    def forward(self, x):
        return self.fc(x)



In [101]:
hlayer = [30, 40]
in_dim = 20
out_dim= 3

myfcn_final = FCN_final(in_dim, hlayer, out_dim)

In [102]:
myfcn_final

FCN_final(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)

In [103]:
# Ordered dict example
# nn.Sequential() example

class FCN_seq_ordered_dic(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc = nn.Sequential(OrderedDict([('lin1', nn.Linear(20, 30)),
                      ('relu1', nn.ReLU(True)),
                      ('lin2', nn.Linear(30, 40)),
                      ('relu2',nn.ReLU(True)),
                      ('lin3', nn.Linear(40, 3))
                                            ])
        )

    def forward(self, x):
        return self.fc(x)



# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [104]:
# ModuleList(), ModuleDict()

In [105]:
# state_dict() example
model.state_dict()

OrderedDict([('lin1.weight',
              tensor([[ 1.1542e-01, -1.5343e-01, -1.4473e-01,  6.0137e-02,  3.4798e-02,
                       -1.9658e-01, -1.6290e-01, -1.7801e-01,  1.7759e-01,  1.4619e-01,
                       -1.1770e-01,  3.3738e-02, -5.1431e-02,  1.9439e-01, -2.0253e-01,
                       -1.6313e-01, -1.8617e-01,  1.3013e-01, -1.1178e-01,  1.3318e-01],
                      [ 7.5586e-02, -1.0930e-01, -6.0237e-02, -9.0490e-02,  9.8650e-02,
                       -8.9674e-02,  7.5885e-02,  1.5974e-02, -1.7857e-01, -1.4056e-01,
                       -1.5504e-01,  1.1717e-01,  4.8451e-02, -5.6224e-03, -1.3314e-01,
                       -2.2155e-01, -7.9131e-02, -1.7114e-01, -2.2241e-01, -2.1327e-01],
                      [-1.4606e-01, -2.1738e-01,  6.9704e-02,  8.3668e-02,  9.0129e-02,
                       -2.2260e-01,  1.3827e-01,  4.3595e-02,  1.1434e-01, -5.4957e-02,
                        1.0738e-01, -2.1207e-01,  7.6162e-02,  1.7731e-01,  9.3997e-02,
 

# Problem Setup

<div style="text-align:center"> <img src='https://drive.google.com/thumbnail?id=1FRhniwGeeutBSJQRdW6GzshMfDrPz7oJ&sz=w1000' width="250" height="200"> </div>
    
Build a Fully connected neural network with

- 3 layers
- 마지막 layer의 unit 수는 `1`
  - 마지막 layer의 activation은 없음 (linear layer)
- Data feature 수는 `100`

- input unit 수는 data 크기를 보고 맞추세요
- hidden layer의 unit 수는 `[80, 50]`
  - hidden layer의 activation 함수는 ReLU

- model class 명 `myFCN`
  - instance 명 `my_model` 생성
  - `my_model` 출력

## Problem 1

problem setup에서 구성한 neural network을 `nn.Sequential`을 활용하여 생성하세요

In [106]:
## 사용할 data
batch_size = 30
num_feature = 100

X_train = torch.randn(batch_size, num_feature)

In [107]:
X_train.shape

torch.Size([30, 100])

In [108]:
# Problem 1 코딩 (매 줄마다 주석 필수 )

# 신경망 클래스 정의
class myFCN(nn.Module):
    def __init__(self):
        super(myFCN, self).__init__()   # nn,Module의 생성자를 호출하여 초기화

        # 신경망의 layer들 순차적으로 정의
        self.fc = nn.Sequential(
            nn.Linear(100, 80),     # 입력층에서 첫 번째 은닉층
            nn.ReLU(inplace=True),  # 첫 번째 은닉층 활성화 함수
            nn.Linear(80, 50),      # 첫 번째에서 두 번째 은닉층
            nn.ReLU(inplace=True),  # 두 번째 은닉층 활성화 함수
            nn.Linear(50, 1)        # 두 번째 은닉층에서 출력층
        )

    def forward(self, x):
        return self.fc(x)     # 입력 데이터 x를 self.fc를 통해 순차적으로 네트워크 레이어를 통과시키고 결과 반환

# myFCN 클래스의 인스턴스를 생성하여 모델 초기화
my_model = myFCN()

# 모델에 데이터를 넣어 결과 확인
output = my_model(X_train)
print(output)

tensor([[-0.1661],
        [-0.1302],
        [-0.1522],
        [-0.0794],
        [-0.1689],
        [-0.1492],
        [-0.1721],
        [-0.2136],
        [-0.2681],
        [-0.1524],
        [-0.1133],
        [-0.0473],
        [-0.0662],
        [-0.1885],
        [-0.2335],
        [-0.1655],
        [-0.1142],
        [-0.1480],
        [-0.0958],
        [-0.0364],
        [-0.2352],
        [-0.0865],
        [-0.1291],
        [-0.1851],
        [-0.0194],
        [-0.1083],
        [-0.0688],
        [-0.2207],
        [-0.0419],
        [-0.1126]], grad_fn=<AddmmBackward0>)


In [109]:
my_model

myFCN(
  (fc): Sequential(
    (0): Linear(in_features=100, out_features=80, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=80, out_features=50, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
)

## Problem 2

problem setup에서 구성한 neural network을 `OrderedDict`을 활용하여 생성하세요
- 각 layer의 이름을 주고 생성하세요

In [110]:
# 답작성

In [111]:
# 신경망 클래스 정의
class myFCN(nn.Module):
    def __init__(self):
        super(myFCN, self).__init__()     # nn.Module의 생성자를 호출하여 기본 설정을 초기화

        # 신경망 layer들 순서대로 정의하기 위해 OrderedDict 사용
        self.fc = nn.Sequential(OrderedDict([
            ('input_to_hidden1', nn.Linear(100, 80)),    # 입력층에서 첫 번째 은닉층
            ('relu1', nn.ReLU(inplace=True)),            # 첫 번째 은닉층 활성화 함수
            ('hidden1_to_hidden2', nn.Linear(80, 50)),   # 첫 번째에서 두 번째 은닉층
            ('relu2', nn.ReLU(inplace=True)),            # 두 번째 은닉층 활성화 함수
            ('hidden2_to_output', nn.Linear(50, 1))      # 두 번째 은닉층에서 출력층
        ]))

    def forward(self, x):
        return self.fc(x)     # 입력 데이터 x를 self.fc를 통해 순차적으로 네트워크 레이어를 통과시키고 결과 반환

# myFCN 클래스의 인스턴스를 생성하여 모델 초기화
my_model = myFCN()

# 모델에 데이터를 넣어 결과 확인
output = my_model(X_train)
print(output)

tensor([[0.1483],
        [0.1826],
        [0.1432],
        [0.2608],
        [0.2090],
        [0.1860],
        [0.2054],
        [0.1414],
        [0.1403],
        [0.1235],
        [0.2688],
        [0.0777],
        [0.1815],
        [0.1306],
        [0.0997],
        [0.0590],
        [0.0728],
        [0.2534],
        [0.1125],
        [0.2235],
        [0.1439],
        [0.2906],
        [0.1000],
        [0.1731],
        [0.1894],
        [0.0844],
        [0.2119],
        [0.1665],
        [0.1242],
        [0.1261]], grad_fn=<AddmmBackward0>)


In [112]:
my_model

myFCN(
  (fc): Sequential(
    (input_to_hidden1): Linear(in_features=100, out_features=80, bias=True)
    (relu1): ReLU(inplace=True)
    (hidden1_to_hidden2): Linear(in_features=80, out_features=50, bias=True)
    (relu2): ReLU(inplace=True)
    (hidden2_to_output): Linear(in_features=50, out_features=1, bias=True)
  )
)